In [97]:
import uuid
import json
import datetime 
import random
import time
import datetime

time_fmt="%Y-%m-%d %H:%M:%S"
minutes = 120
start_time = datetime.datetime(2018, 1, 1, 8, 0)
categories = 10
dataset = []

def mk_hrs(start_time):
    hrs = []
    for m in range(minutes):
        hr = round(60 + (40 * random.random()))
        hrs.append(hr)
    return hrs

for cat in range(categories):
    hr_data = mk_hrs(start_time)
    item = {"start": start_time.strftime(time_fmt), "target": hr_data , "cat": cat}
    dataset.append(item)
    
print(dataset[1])


{'cat': 1, 'target': [100, 60, 97, 67, 73, 80, 69, 78, 76, 74, 98, 66, 61, 70, 100, 89, 85, 99, 64, 86, 81, 89, 82, 85, 70, 99, 88, 61, 80, 98, 65, 75, 98, 88, 99, 90, 98, 89, 78, 95, 80, 72, 89, 88, 61, 71, 94, 74, 67, 85, 62, 65, 85, 96, 85, 74, 72, 74, 65, 79, 91, 67, 64, 79, 63, 83, 97, 78, 62, 82, 99, 88, 83, 66, 77, 73, 60, 76, 60, 62, 92, 88, 78, 98, 64, 79, 88, 83, 87, 60, 82, 66, 87, 61, 80, 96, 91, 84, 72, 78, 73, 99, 79, 61, 94, 74, 84, 82, 74, 77, 84, 92, 76, 69, 86, 64, 91, 84, 92, 94], 'start': '2018-01-01 08:00:00'}


In [98]:
random.shuffle(dataset)
split = round(len(dataset) * 0.7)
training = dataset[:split]
test = dataset[split:]
print (len(training))
print (len(test))


7
3


In [99]:
import uuid
uuid = str(uuid.uuid4())

In [100]:
bucket         = 'hrdata_'+uuid
prefix         = 'sagemaker'
region         = "us-east-1" 
train_key      = 'deepar_training.json'
test_key       = 'deepar_test.json'
train_prefix   = '{}/{}'.format(prefix, 'train')
test_prefix    = '{}/{}'.format(prefix, 'test')
output_prefix  = '{}/{}'.format(prefix, 'output')
output_path = 's3://{}/{}'.format(bucket, output_prefix)

In [101]:
def write_datafile(list,file):
    fh = open(file, "w")
    for data in list:
        print(json.dumps(data),file=fh)
    fh.close

write_datafile(training,train_key)
write_datafile(test,test_key)

!wc -l {train_key}
!wc -l {test_key}

       7 deepar_training.json
       3 deepar_test.json


In [102]:
!head {train_key}

{"cat": 3, "target": [76, 61, 97, 92, 67, 99, 84, 73, 99, 77, 98, 67, 89, 93, 91, 75, 72, 62, 87, 88, 61, 62, 74, 90, 73, 79, 73, 85, 69, 89, 72, 67, 71, 85, 90, 89, 91, 68, 81, 76, 76, 94, 66, 93, 83, 71, 61, 99, 73, 93, 88, 94, 63, 69, 83, 99, 67, 76, 93, 96, 72, 83, 62, 69, 84, 96, 76, 91, 68, 70, 86, 100, 76, 89, 62, 66, 92, 73, 78, 80, 84, 85, 81, 74, 86, 62, 78, 77, 90, 76, 84, 68, 99, 65, 65, 87, 93, 99, 70, 91, 80, 88, 87, 97, 85, 90, 78, 71, 88, 63, 89, 88, 90, 81, 84, 80, 98, 90, 88, 71], "start": "2018-01-01 08:00:00"}
{"cat": 8, "target": [75, 62, 69, 83, 91, 62, 61, 63, 80, 68, 76, 71, 65, 69, 84, 77, 64, 85, 93, 85, 85, 98, 79, 75, 76, 77, 70, 71, 96, 71, 80, 95, 92, 85, 97, 78, 68, 62, 76, 92, 61, 95, 67, 62, 69, 73, 83, 82, 93, 84, 99, 77, 82, 67, 63, 91, 82, 80, 78, 70, 89, 97, 71, 70, 77, 92, 80, 78, 75, 70, 87, 86, 69, 88, 99, 70, 72, 91, 80, 78, 91, 68, 81, 78, 65, 65, 98, 95, 65, 98, 88, 69, 73, 80, 91, 89, 67, 75, 72, 67, 83, 73, 81, 82, 78, 90, 63, 72, 62, 84, 81

In [103]:
!aws s3 mb s3://{bucket}

make_bucket: hrdata_aa56b284-6ad1-4d9f-b5c4-e04c36ae2902


In [104]:
import sagemaker
sagemaker_session = sagemaker.Session()

In [105]:
train_path  = sagemaker_session.upload_data(train_key, bucket=bucket, key_prefix=train_prefix)
test_path   = sagemaker_session.upload_data(test_key,  bucket=bucket, key_prefix=test_prefix)

In [106]:
!aws s3 ls --recursive s3://{bucket}/sagemaker/

2018-02-04 17:50:11       1608 sagemaker/test/deepar_test.json
2018-02-04 17:50:10       3753 sagemaker/train/deepar_training.json


In [107]:
containers = {
    'us-east-1': '522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:latest',
    'us-east-2': '566113047672.dkr.ecr.us-east-2.amazonaws.com/forecasting-deepar:latest',
    'us-west-2': '156387875391.dkr.ecr.us-west-2.amazonaws.com/forecasting-deepar:latest',
    'eu-west-1': '224300973850.dkr.ecr.eu-west-1.amazonaws.com/forecasting-deepar:latest'
}

image_name = containers[region]
image_name

'522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:latest'

In [108]:
role_name="sagemaker_role_{}".format(uuid)
role_arn = !aws iam create-role \
    --role-name "{role_name}" \
    --assume-role-policy-document "file://sagemaker-trust.json" \
    --query Role.Arn \
    --output text
    
role_arn = role_arn.s
role_arn

'arn:aws:iam::030555009967:role/sagemaker_role_aa56b284-6ad1-4d9f-b5c4-e04c36ae2902'

In [109]:
!aws iam attach-role-policy \
    --role-name {role_name} \
    --policy-arn "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess"

In [110]:
!aws iam attach-role-policy \
    --role-name {role_name} \
    --policy-arn "arn:aws:iam::aws:policy/AmazonS3FullAccess"

In [115]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role_arn,
    train_instance_count=1,
    train_instance_type='ml.c4.8xlarge',
    base_job_name='bp-job',
    output_path=output_path
)

In [116]:
prediction_length = 30
hyperparameters = {
    "time_freq": 'M',
    "context_length": prediction_length,
    "prediction_length": prediction_length, # number of data points to predict
    "num_cells": "40",
    "num_layers": "2",
    "likelihood": "gaussian",
    "epochs": "250",
    "mini_batch_size": "32",
    "learning_rate": "0.0001",
    "dropout_rate": "0.05",
    "early_stopping_patience": "10" # stop if loss hasn't improved in 10 epochs
}

estimator.set_hyperparameters(**hyperparameters)

In [117]:
data_channels = {"train": train_path, "test": test_path}
data_channels

{'test': 's3://hrdata_aa56b284-6ad1-4d9f-b5c4-e04c36ae2902/sagemaker/test/deepar_test.json',
 'train': 's3://hrdata_aa56b284-6ad1-4d9f-b5c4-e04c36ae2902/sagemaker/train/deepar_training.json'}

In [118]:
estimator.fit(inputs=data_channels)

INFO:sagemaker:Creating training-job with name: bp-job-2018-02-04-16-51-44-527


.....................................................................
Arguments: train
[02/04/2018 16:57:35 INFO 139784009750336] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'dropout_rate': u'0.05', u'cardinality': u'', u'test_quantiles': u'[0.5, 0.9]', u'_num_gpus': u'auto', u'learning_rate': u'0.001', u'_kvstore': u'auto', u'num_layers': u'3', u'embedding_dimension': u'', u'num_cells': u'40', u'_num_kv_servers': u'auto', u'mini_batch_size': u'32', u'likelihood': u'gaussian', u'early_stopping_patience': u''}
[02/04/2018 16:57:35 INFO 139784009750336] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'dropout_rate': u'0.05', u'learning_rate': u'0.0001', u'num_cells': u'40', u'prediction_length': u'30', u'epochs': u'250', u'time_freq': u'M', u'context_length': u'30', u'num_layers': u'2', u'mini_batch_size': u'32', u'likelihood': u'gaussian', u'early_stopping_patience': u'10'}
[02/04/2018 16:57:

[02/04/2018 16:57:46 INFO 139784009750336] processed a total of 320 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1139.9478912353516, "sum": 1139.9478912353516, "min": 1139.9478912353516}}, "EndTime": 1517763466.679978, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1517763465.526533}

[02/04/2018 16:57:46 INFO 139784009750336] loss did not improve for 3 epochs
[02/04/2018 16:57:46 INFO 139784009750336] Epoch[8] Batch[0] avg_epoch_loss=5.201490
[02/04/2018 16:57:47 INFO 139784009750336] Epoch[8] Batch[5] avg_epoch_loss=23.701459
[02/04/2018 16:57:47 INFO 139784009750336] Epoch[8] Batch [5]#011Speed: 256.55 samples/sec#011loss=23.701459
[02/04/2018 16:57:47 INFO 139784009750336] processed a total of 320 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1116.1890029907227, "sum": 1116.1890029907227, "min": 1116.1890029907227}}, "EndTime": 1517763467.810856, "Dimensions": {"Host": "algo-1", "Operation"

===== Job Complete =====


In [119]:
job_name = estimator.latest_training_job.name
job_name

'bp-job-2018-02-04-16-51-44-527'

In [121]:
endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    deployment_image=image_name,
    role=role_name
)

INFO:sagemaker:Creating model with name: bp-job-2018-02-04-16-51-44-527
INFO:sagemaker:Creating endpoint-config with name bp-job-2018-02-04-16-51-44-527
INFO:sagemaker:Creating endpoint with name bp-job-2018-02-04-16-51-44-527


------------------------------------------------------------------------------------------------------------------------------------!

In [ ]:
q1 = '0.1'         # compute p10 quantile
q2 = '0.9'         # compute p90 quantile
num_samples = 5    # predict 6 sample series
target = test[0]

s = {"start": "2017-02-01 08:00:00", "target": target}
series = []
series.append(s)

configuration = {
        "output_types": ["samples"],
        "num_samples": num_samples}
http_data = {
        "instances": series, 
        "configuration": configuration
}
http_data = json.dumps(http_data)
print(http_data)